In [1]:
from glob import glob
import os
import mne 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
 # no of epochs / no of channels/ length of epochs

In [2]:
files=glob("EEG files folder/*.edf")

In [3]:
len(files)

40

In [8]:
def read_extract(file_path):
    raw = mne.io.read_raw_edf(file_path, preload=True)
    # raw.filter(l_freq=0.5,h_freq=35)
    print(f"###########################{file_path}##############################################")
    channel_names = ['FP1', 'FP2', 'F7', 'F8']
    raw = raw.copy().pick_channels(channel_names)
    max_time = raw.times[-1]
    tmax1=min(max_time,600)
    tmax2=min(max_time,1860)
    tmin1=min(300,max_time)
    tmin2=min(1500,max_time)
    no_stress_interval = raw.copy().crop(tmin=tmin1, tmax=tmax1)
    epochs_no_stress=mne.make_fixed_length_epochs(no_stress_interval,duration=1,overlap=0.5)
    array_no_streesed=epochs_no_stress.get_data()

    stressed_interval = raw.copy().crop(tmin=tmin2, tmax=tmax2)
        
    epochs_stressed=mne.make_fixed_length_epochs(stressed_interval,duration=1,overlap=0.5)
    array_streesed=epochs_stressed.get_data()
    return array_no_streesed,array_streesed

In [9]:

no_stressed_data=[]
stressed_data=[]
for i in files:
    no_stressed,stressed=read_extract(i)
    no_stressed_data.append(no_stressed)
    stressed_data.append(stressed)

Extracting EDF parameters from c:\Users\itsay\Desktop\Decision Lab\EEG files folder\0000_20170802_095129.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1368149  =      0.000 ...  5472.596 secs...
###########################EEG files folder\0000_20170802_095129.edf##############################################
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Not setting metadata
599 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 599 events and 250 original time points ...
0 bad epochs dropped
Not setting metadata
719 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 719 events and 250 original time points ...
0 bad epochs dropped
Extracting EDF parameters from c:\Users\itsay\Desktop\Decision Lab\EEG files folder\0_20170721_112936.edf...
EDF file detected
Setting chann

In [10]:
len(no_stressed_data)

40

In [11]:
len(stressed_data)

40

In [12]:
no_stressed_data[2].shape

(599, 4, 250)

In [13]:
stressed_data[0].shape

(719, 4, 250)

In [14]:
no_stressed_epoch_labels=[len(i)*[0] for i in no_stressed_data]
stressed_epoch_labels=[len(i)*[1] for i in stressed_data]

In [15]:
len(no_stressed_epoch_labels),len(stressed_epoch_labels)

(40, 40)

In [16]:
data_list=no_stressed_data+stressed_data
label_list=no_stressed_epoch_labels+stressed_epoch_labels

In [17]:
group_list =[[i]*len(j) for i,j in enumerate(data_list)]
len(group_list)

80

In [18]:
data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
group_array=np.hstack(group_list)
print(data_array.shape,label_array.shape,group_array.shape)

(52720, 4, 250) (52720,) (52720,)


PART 2 USING LOGISTIC REGRESSION

In [19]:
from scipy import stats
def mean(x):
    return np.mean(x,axis=-1)
def std(x):
    return np.std(x,axis=-1)
def ptp(x):
    return np.ptp(x,axis=-1)
def var(x):
    return np.var(x,axis=-1)
def maxim(x):
    return np.max(x,axis=-1)
def argminim(x):
    return np.argmin(x,axis=-1)
def argmaxim(x):
    return np.argmax(x,axis=-1)
def minim(x):
     return np.min(x,axis=-1)
    
def concatenate_features(x):
    return np.concatenate((mean(x),std(x),var(x),ptp(x),minim(x),maxim(x),argmaxim(x),argminim(x)),axis=-1)


In [20]:
features=[]
for d in data_array:
    features.append((concatenate_features(d)))

In [21]:
features=np.array(features)
features.shape

(52720, 32)

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV

In [31]:
clf=LogisticRegression()
gkf=GroupKFold(5)
pipe=Pipeline([('scaler',StandardScaler()),('clf',clf )])
param_grid={'clf__C': [0.1,0.5,0.7,1,3,5,7]}
gscv=GridSearchCV(pipe,param_grid,cv=gkf,n_jobs=12)
gscv.fit(features,label_array,groups=group_array)

GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('clf', LogisticRegression())]),
             n_jobs=12, param_grid={'clf__C': [0.1, 0.5, 0.7, 1, 3, 5, 7]})

In [24]:
gscv.best_score_

0.8354135053110774

PART 3 CNN VIA TENSORFLOW

In [25]:
data_array.shape,label_array.shape,group_array.shape

((52720, 4, 250), (52720,), (52720,))

In [26]:
data_array=np.moveaxis(data_array,1,2)
data_array.shape

(52720, 250, 4)

In [27]:
from tensorflow.keras.layers import Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,\
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
def cnnmodel():
    clear_session()
    model=Sequential()
    model.add(Conv1D(filters=5,kernel_size=3,strides=1,input_shape=(250,4)))#1
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#2
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#3
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#4
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#5
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#6
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#7
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#8
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#9
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())#10
    model.add(Dense(1,activation='sigmoid'))#11
    
    model.compile('adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

model=cnnmodel()
model.summary()

c:\Users\itsay\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 248, 5)         │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 248, 5)         │            20 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 248, 5)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 124, 5)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 122, 5)         │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 122, 5)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 61, 5)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 61, 5)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 59, 5)          │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 59, 5)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d               │ (None, 29, 5)          │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 29, 5)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 27, 5)          │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 27, 5)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d_1             │ (None, 13, 5)          │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 11, 5)          │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 11, 5)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 5)              │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 411 (1.61 KB)

 Trainable params: 401 (1.57 KB)

 Non-trainable params: 10 (40.00 B)

In [32]:
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gfk=GroupKFold()

In [34]:
accuracy=[]
for train_index, val_index in gkf.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    model=cnnmodel()
    model.fit(train_features,train_labels,epochs=10,batch_size=128,validation_data=(val_features,val_labels))
    accuracy.append(model.evaluate(val_features,val_labels)[1])

c:\Users\itsay\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.7859 - loss: 0.5231 - val_accuracy: 0.8146 - val_loss: 0.5341
Epoch 2/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.8803 - loss: 0.3030 - val_accuracy: 0.8471 - val_loss: 0.4588
Epoch 3/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8840 - loss: 0.2863 - val_accuracy: 0.8299 - val_loss: 0.4410
Epoch 4/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8804 - loss: 0.2965 - val_accuracy: 0.8445 - val_loss: 0.4387
Epoch 5/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8779 - loss: 0.2991 - val_accuracy: 0.8428 - val_loss: 0.4139
Epoch 6/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8830 - loss: 0.2819 - val_accuracy: 0.8377 - val_loss: 0.4187
Epoch 7/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.8808 - loss: 0.2869 - val_accuracy: 0.8323 - val_loss: 0.4748
Epoch 8/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 22s 67ms/step - accuracy: 0.8802 - loss: 0.2922 - val_accura

c:\Users\itsay\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.7133 - loss: 0.5727 - val_accuracy: 0.8481 - val_loss: 0.3575
Epoch 2/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8679 - loss: 0.3502 - val_accuracy: 0.8622 - val_loss: 0.3291
Epoch 3/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8684 - loss: 0.3381 - val_accuracy: 0.8742 - val_loss: 0.3221
Epoch 4/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8705 - loss: 0.3211 - val_accuracy: 0.8864 - val_loss: 0.3124
Epoch 5/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8674 - loss: 0.3286 - val_accuracy: 0.8720 - val_loss: 0.3230
Epoch 6/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8694 - loss: 0.3232 - val_accuracy: 0.8858 - val_loss: 0.3155
Epoch 7/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8708 - loss: 0.3165 - val_accuracy: 0.8814 - val_loss: 0.3187
Epoch 8/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8685 - loss: 0.3228 - val_accuracy: 

c:\Users\itsay\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.7831 - loss: 0.5789 - val_accuracy: 0.9323 - val_loss: 0.2321
Epoch 2/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8387 - loss: 0.3868 - val_accuracy: 0.9128 - val_loss: 0.2365
Epoch 3/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 23s 70ms/step - accuracy: 0.8366 - loss: 0.3812 - val_accuracy: 0.9052 - val_loss: 0.2464
Epoch 4/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 19s 57ms/step - accuracy: 0.8382 - loss: 0.3716 - val_accuracy: 0.8874 - val_loss: 0.2895
Epoch 5/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.8392 - loss: 0.3741 - val_accuracy: 0.9075 - val_loss: 0.2356
Epoch 6/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8382 - loss: 0.3734 - val_accuracy: 0.8973 - val_loss: 0.2591
Epoch 7/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8389 - loss: 0.3683 - val_accuracy: 0.9174 - val_loss: 0.2393
Epoch 8/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8363 - loss: 0.3708 - val_accur

c:\Users\itsay\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 18s 34ms/step - accuracy: 0.7796 - loss: 0.6103 - val_accuracy: 0.9136 - val_loss: 0.4288
Epoch 2/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - accuracy: 0.8569 - loss: 0.3598 - val_accuracy: 0.9228 - val_loss: 0.6485
Epoch 3/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - accuracy: 0.8570 - loss: 0.3555 - val_accuracy: 0.9229 - val_loss: 0.4537
Epoch 4/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.8566 - loss: 0.3503 - val_accuracy: 0.9219 - val_loss: 0.5221
Epoch 5/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8569 - loss: 0.3479 - val_accuracy: 0.9213 - val_loss: 0.4638
Epoch 6/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8589 - loss: 0.3446 - val_accuracy: 0.9222 - val_loss: 0.4858
Epoch 7/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8576 - loss: 0.3529 - val_accuracy: 0.9223 - val_loss: 0.4747
Epoch 8/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8586 - loss: 0.3410 - val_accur

c:\Users\itsay\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8930 - loss: 0.4704 - val_accuracy: 0.7484 - val_loss: 0.9429
Epoch 2/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8972 - loss: 0.3374 - val_accuracy: 0.7555 - val_loss: 0.5585
Epoch 3/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8977 - loss: 0.3015 - val_accuracy: 0.7547 - val_loss: 0.5541
Epoch 4/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8985 - loss: 0.2937 - val_accuracy: 0.7531 - val_loss: 0.6012
Epoch 5/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8947 - loss: 0.2961 - val_accuracy: 0.7560 - val_loss: 0.5262
Epoch 6/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8991 - loss: 0.2801 - val_accuracy: 0.7497 - val_loss: 0.5072
Epoch 7/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8948 - loss: 0.2864 - val_accuracy: 0.7516 - val_loss: 0.5398
Epoch 8/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8995 - loss: 0.2796 - val_accuracy: 0

In [35]:
np.mean(accuracy)

0.8514415740966796